In [59]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from typing import TypedDict, Annotated
from dotenv import load_dotenv
import operator

In [60]:
load_dotenv()

True

In [61]:
model = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0
)

In [62]:
class Eval_schema(BaseModel):
    feedback: str = Field(description='detailed feedback of essay covering all mistakes')
    score: int = Field(description='score out of 10', ge=0 , le=10)

In [63]:
structured_model = model.with_structured_output(Eval_schema)

In [64]:
class Essay_state(TypedDict):

    essay_text: str

    lang_feedback: str
    depth_feedback: str
    clarity_feedback: str
    relevance_feedback: str
    evidence_feedback: str

    overall_feedback: str
    avg_score: float

    scores: Annotated[list[float] , operator.add]


In [65]:
def eval_lang(state:Essay_state) -> Essay_state:

    template = PromptTemplate(
        template='Evaluate the following essay for language quality. Comment on grammar, vocabulary, sentence structure, and tone. Give constructive feedback in 3–4 sentences. \n text:{essay_text}',
        input_variables=['essay_text']
    )

    prompt = template.invoke({'essay_text':state['essay_text']})
    response = structured_model.invoke(prompt)

    return {'lang_feedback':response.feedback , 'scores':[response.score]}

def eval_depth(state:Essay_state) -> Essay_state:

    template = PromptTemplate(
        template='Evaluate the following essay for depth of analysis. Does it show critical thinking, originality, and nuanced reasoning? Point out strengths and areas for improvement in 3–4 sentences. \n text:{essay_text}',
        input_variables=['essay_text']
    )

    prompt = template.invoke({'essay_text':state['essay_text']})
    response = structured_model.invoke(prompt)

    return {'depth_feedback':response.feedback , 'scores':[response.score]}

def eval_clarity(state:Essay_state) -> Essay_state:

    template = PromptTemplate(
        template='Evaluate the following essay for clarity. Focus on organization, logical flow, and ease of understanding. Provide specific feedback in 3–4 sentences. \n text:{essay_text}',
        input_variables=['essay_text']
    )

    prompt = template.invoke({'essay_text':state['essay_text']})
    response = structured_model.invoke(prompt)

    return {'clarity_feedback':response.feedback , 'scores':[response.score]}

def eval_relevance(state:Essay_state) -> Essay_state:

    template = PromptTemplate(
        template='Evaluate the following essay for relevance to the topic. Does it stay on point or deviate into unnecessary areas? Provide concise feedback in 3–4 sentences. \n text:{essay_text}',
        input_variables=['essay_text']
    )

    prompt = template.invoke({'essay_text':state['essay_text']})
    response = structured_model.invoke(prompt)

    return {'relevance_feedback':response.feedback , 'scores':[response.score]}

def eval_evidence(state:Essay_state) -> Essay_state:

    template = PromptTemplate(
        template='Evaluate the following essay for use of evidence. Does it support claims with examples, facts, or references? Suggest improvements in 3–4 sentences. \n text:{essay_text}',
        input_variables=['essay_text']
    )

    prompt = template.invoke({'essay_text':state['essay_text']})
    response = structured_model.invoke(prompt)

    return {'evidence_feedback':response.feedback , 'scores':[response.score]}


In [66]:
def gen_summary(state:Essay_state) -> Essay_state:

    template = PromptTemplate(
        template="Given the essay {essay_text} and the feedbacks [Language: {lang_feedback}; Depth: {depth_feedback}; Clarity: {clarity_feedback}; Relevance: {relevance_feedback}; Evidence: {evidence_feedback}], write a balanced 4–6 sentence summary feedback integrating all points without repeating the parameter names.",
        input_variables=['essay_text','lang_feedback','depth_feedback','clarity_feedback','relevance_feedback','evidence_feedback']
    )
    prompt = template.invoke({
        'essay_text': state['essay_text'],
        'lang_feedback': state['lang_feedback'],
        'depth_feedback': state['depth_feedback'],
        'clarity_feedback': state['clarity_feedback'],
        'relevance_feedback': state['relevance_feedback'],
        'evidence_feedback': state['evidence_feedback']
    })

    summary_feedback = model.invoke(prompt).content
    avg_score = sum(state['scores']) / len(state['scores'])

    return {'overall_feedback':summary_feedback, 'avg_score':avg_score}


In [67]:
graph = StateGraph(Essay_state)

graph.add_node('eval_lang',eval_lang)
graph.add_node('eval_depth',eval_depth)
graph.add_node('eval_clarity',eval_clarity)
graph.add_node('eval_relevance',eval_relevance)
graph.add_node('eval_evidence',eval_evidence)
graph.add_node('gen_summary',gen_summary) 

graph.add_edge(START,'eval_lang')
graph.add_edge(START,'eval_depth')
graph.add_edge(START,'eval_clarity')
graph.add_edge(START,'eval_relevance')
graph.add_edge(START,'eval_evidence')

graph.add_edge('eval_lang','gen_summary')
graph.add_edge('eval_depth','gen_summary')
graph.add_edge('eval_clarity','gen_summary')
graph.add_edge('eval_relevance','gen_summary')
graph.add_edge('eval_evidence','gen_summary')

graph.add_edge('gen_summary',END)

workflow = graph.compile()

In [70]:
essay = """The Importance of Critical Thinking in the Digital Age.
We live in an age overflowing with information: newsfeeds, social media posts, opinion pieces, and instant commentary appear every second. This abundance brings huge benefits — easier access to learning, faster communication, and broadened perspectives — but it also creates new challenges. Chief among them is the need for strong critical thinking skills. Critical thinking helps people separate reliable information from noise, evaluate sources and arguments, and make decisions grounded in reason rather than impulse or popularity.
A critical thinker questions assumptions instead of accepting them automatically. For example, when encountering a striking claim online, such a person will check the provenance of the claim, look for supporting evidence, and ask whether alternative explanations exist. This attitude reduces the spread of misinformation and prevents poor decisions that arise from incomplete or biased information. In academic and professional settings, critical thinking enables deeper analysis, better problem-solving, and more persuasive arguments — all crucial skills in competitive environments.
Developing critical thinking also fosters intellectual humility. It encourages individuals to acknowledge the limits of their knowledge, seek diverse perspectives, and update beliefs when presented with better evidence. In polarizing environments, this habit promotes respectful dialogue and reduces echo chambers where ideas are blindly reinforced. Moreover, in practical terms, critical thinking improves everyday tasks: choosing trustworthy news sources, assessing product reviews, or understanding statistical claims in public policy debates.
Education systems and workplaces should therefore emphasize reasoning, media literacy, and evidence-based discussion. Teaching students to evaluate arguments, interpret data, and construct clear justifications prepares them for civic life and the job market. Employers who value these skills gain teams that can adapt, innovate, and make sound decisions under uncertainty.
In short, critical thinking is not an optional intellectual luxury — it is a practical necessity in the digital era. By cultivating skepticism tempered with openness, individuals can navigate information complexity, make informed choices, and contribute to healthier public discourse."""

essay = """The moon is basically made of cheese and everyone knows that because cows jump over it in the nursery rhyme which is like basically evidence. Social media is the real school of life since scrolling teaches patience when WiFi is slow and anger management when it crashes. Teachers should be replaced with holograms of cats because cats are wise and mysterious, plus they never mark exams unfairly. Also, history is just a conspiracy by textbook companies who want to sell more paper, but digital tablets are spying on us through emojis. Critical thinking is overrated because if you think too much, you forget to vibe with the universe’s natural TikTok rhythms. In conclusion, life is like a sandwich: sometimes soggy, sometimes crunchy, but always confusing if you don’t put enough ketchup on it."""


initial_state = {
    'essay_text':essay
}

output_state = workflow.invoke(initial_state)

In [71]:
output_state

{'essay_text': 'The moon is basically made of cheese and everyone knows that because cows jump over it in the nursery rhyme which is like basically evidence. Social media is the real school of life since scrolling teaches patience when WiFi is slow and anger management when it crashes. Teachers should be replaced with holograms of cats because cats are wise and mysterious, plus they never mark exams unfairly. Also, history is just a conspiracy by textbook companies who want to sell more paper, but digital tablets are spying on us through emojis. Critical thinking is overrated because if you think too much, you forget to vibe with the universe’s natural TikTok rhythms. In conclusion, life is like a sandwich: sometimes soggy, sometimes crunchy, but always confusing if you don’t put enough ketchup on it.',
 'lang_feedback': 'The essay contains frequent grammatical errors, such as missing articles and incorrect verb forms, which disrupt readability. Vocabulary choices are often informal or